In [1]:
!pip install psycopg2-binary

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 330.3 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.7 MB 751.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.7 MB 1.2 MB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.7 MB 1.6 MB/s eta 0:00:02
   ------ --------------------------------- 0.5/2.7 MB 1.8 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.7 MB 2.4 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.7 MB 2.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.4/2.7 MB 3.7 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 3.6 MB/s eta 0:00:01
   ----------------------------- ---------- 2.0/2.7 MB 4.2 MB/s eta 0:00:01
   -------------------------------- ------- 2.2/2.7 MB 4.1 MB/s eta 0:00:01
   -------------------

In [42]:
from typing import List, Literal, Optional, Dict
from pydantic import BaseModel, Field


class PlannerStep(BaseModel):
    step_id: str = Field(
        ...,
        description="Краткий идентификатор шага, например 'plan', 'classify', 'collect', 'analyze'"
    )
    description: str = Field(
        ...,
        description="Человеко-читаемое описание шага"
    )
    agent: Literal["Planner", "Classifier", "Collector", "Analyst"] = Field(
        ...,
        description="Какой агент выполняет шаг"
    )
    input_from: List[str] = Field(
        ...,
        description="ID предыдущих шагов, чьи результаты используются на этом шаге"
    )


class PlannerOutput(BaseModel):
    # 1. Понимание задачи
    user_intent: Literal[
        "describe_dataset",          # просто описать, что есть в данных
        "trend_over_time",          # динамика по годам / времени
        "compare_groups",           # сравнение групп (например, стран, доменов)
        "top_k_entities",           # топ-k авторов / журналов / стран и т.п.
        "correlation_or_relationship",  # связь между параметрами (например, цитируемость vs год)
        "raw_table_view",           # просто таблица (raw data / отфильтрованный срез)
        "other",
    ] = Field(
        ...,
        description="Высокоуровневый тип анализа, который запросил пользователь"
    )

    question_rewrite: str = Field(
        ...,
        description="Переформулировка запроса пользователя в строгих технических терминах"
    )

    # 2. Нужен ли Classifier и по какой оси группировать
    requires_classifier: bool = Field(
        ...,
        description="Нужно ли вызывать Classifier для выбора конкретных значений параметров из словарей БД"
    )

    classification_dimension: Optional[Literal[
        "domain",
        "field",
        "subfield",
        "topic",
        "title",
        "publication_year",
        "cited_by_count",
        "journal",
        "abstract",
        "doi",
        "none",
    ]] = Field(
        None,
        description="Список полей из таблицы articles, для которых Classifier должен выбрать подходящие значения из известных списков (словари допустимых значений)"
    )

    # 3. Фильтры, которые Collector должен учесть в SQL
    filters: List[str] = Field(
        default_factory=list,
        description=(
            "Набор текстовых фильтров, которые нужно отразить в SQL "
            "(годы, домены, страны, ограничения по цитируемости и т.п.)"
        ),
    )

     # 4. Инструкция для Collector’а: SQL-шаблон и параметры
    collector_sql_template: str = Field(
        ...,
        description=(
            "Параметризованный SQL-шаблон для Collector’а. "
            "Используются только реальные поля таблицы articles: "
            "id, title, doi, publication_year, abstract, citated_by_count, "
            "journal, domain, field, subfield, topic. "
            "Вместо конкретных значений – плейсхолдеры вида :start_year, :end_year, :topics и т.п."
        ),
    )

    collector_parameters: Dict[str, str] = Field(
        default_factory=dict,
        description=(
            "Словарь параметр → описание. "
            "Ключи – имена плейсхолдеров без двоеточия (например, 'start_year', 'topics'). "
            "Значения – текстовое объяснение, ЧТО Classifier должен туда подставить "
            "(например, 'Начальный год диапазона publication_year, выбранный из списка всех доступных годов')."
        ),
    )

    # 4.1 Явный список ожидаемых параметров (удобно для Classifier’а)
    required_parameters: List[str] = Field(
        default_factory=list,
        description=(
            "Список имён параметров (без двоеточия), которые должны быть заполнены "
            "Classifier’ом перед выполнением SQL. "
            "Например: ['start_year', 'end_year', 'topics']."
        ),
    )

    # 5. Формат результата и задачи для Analyst’а
    expected_result_format: Literal[
        "table",
        "bar_chart",
        "line_chart",
        "histogram",
        "text_summary",
        "table_and_summary",
        "chart_and_summary",
    ] = Field(
        ...,
        description="В каком виде Analyst должен представить итоговый результат"
    )

    analyst_instructions: str = Field(
        ...,
        description="Чёткая инструкция для Analyst’а: что посчитать, как объяснить, что визуализировать"
    )

    # 6. План шагов между агентами (для LangGraph / оркестратора)
    steps: List[PlannerStep] = Field(
        ...,
        description="Декомпозиция задачи на шаги для Planner / Classifier / Collector / Analyst"
    )

In [44]:
from typing import Any, Dict, List, Literal, Union, Optional
from pydantic import BaseModel, Field


class MatchedValues(BaseModel):
    column: Literal["publication_year", "domain", "field", "subfield", "topic", "journal"]
    values: List[Union[int, str]] = Field(
        default_factory=list,
        description="Список значений из словаря по этому столбцу, которые соответствуют запросу пользователя"
    )
    rationale: Optional[str] = Field(
        None,
        description="Краткое объяснение, почему выбраны именно эти значения (для отладки/логов)"
    )


class ClassifierOutput(BaseModel):
    used_query: str = Field(
        ...,
        description="Фактический текст запроса, на основе которого принималось решение (можно взять question_rewrite)"
    )

    # Параметры, которые будут подставлены в SQL-шаблон Collector'а
    resolved_parameters: Dict[str, Union[int, str, List[Union[int, str]]]] = Field(
        ...,
        description=(
            "Словарь параметр → значение, которые нужно подставить в SQL-шаблон. "
            "Ключи ДОЛЖНЫ совпадать с именами из PlannerOutput.required_parameters."
        ),
    )

    # Для прозрачности – какие значения по каким полям были подобраны
    matched_values: List[MatchedValues] = Field(
        default_factory=list,
        description=(
            "Список подобранных значений по каждому столбцу (publication_year/domain/field/subfield/topic/journal) "
            "из известных словарей."
        ),
    )

    # На всякий случай – флаг уверенности
    confidence: float = Field(
        0.0,
        description="Эвристическая оценка уверенности Classifier'а от 0 до 1"
    )

In [46]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field


class ClassifierContext(BaseModel):
    """Контекст, который мы передаём в LLM-классификатор.

    Здесь уже есть:
      - запрос пользователя и переписанный вопрос от Planner'а,
      - какие параметры нужно заполнить (required_parameters),
      - по каким полям таблицы нужно подбирать значения (classification_dimensions),
      - возможные значения по каждому полю (possible_*),
      - SQL-шаблон и описание параметров Collector'а.
    """

    user_query: str
    question_rewrite: Optional[str]
    required_parameters: List[str]
    classification_dimensions: List[str]

    possible_publication_years: List[int] = []
    possible_domains: List[str] = []
    possible_fields: List[str] = []
    possible_subfields: List[str] = []
    possible_topics: List[str] = []
    possible_journals: List[str] = []

    collector_sql_template: str
    collector_parameters: Dict[str, str]


def build_classifier_context(
    user_query: str,
    planner_output: PlannerOutput,
    possible_values: Dict[str, list],
) -> ClassifierContext:
    """Собрать ClassifierContext на основе PlannerOutput и словарей значений из БД."""

    # В текущей версии PlannerOutput у тебя есть одиночное поле classification_dimension.
    # Для Classifier'а удобнее работать со списком, поэтому оборачиваем его в список,
    # если оно не равно None/"none".
    dims: List[str] = []
    dim_single = getattr(planner_output, "classification_dimension", None)
    if dim_single and dim_single != "none":
        dims = [dim_single]

    return ClassifierContext(
        user_query=user_query,
        question_rewrite=planner_output.question_rewrite,
        required_parameters=planner_output.required_parameters,
        classification_dimensions=dims,
        possible_publication_years=possible_values.get("publication_year", []),
        possible_domains=possible_values.get("domain", []),
        possible_fields=possible_values.get("field", []),
        possible_subfields=possible_values.get("subfield", []),
        possible_topics=possible_values.get("topic", []),
        possible_journals=possible_values.get("journal", []),
        collector_sql_template=planner_output.collector_sql_template,
        collector_parameters=planner_output.collector_parameters,
    )

In [48]:
from typing import Any, Dict, List


def adapt_planner_json(raw_obj: Dict[str, Any]) -> Dict[str, Any]:
    obj: Dict[str, Any] = dict(raw_obj)

    # Если это JSON Schema, а не план — явно падаем с понятной ошибкой
    if "properties" in obj and "user_intent" not in obj and "type" in obj and obj.get("type") == "object":
        raise RuntimeError(
            "Модель вернула JSON Schema (описание PlannerOutput), "
            "а не конкретный план. Нужно поправить system prompt: "
            "запретить JSON Schema и попросить вернуть объект с заполненными полями."
        )

    # type -> user_intent (это для старого формата с полем "type": "describe_dataset" и т.п.)
    if "user_intent" not in obj and "type" in obj:
        obj["user_intent"] = obj.pop("type")

    obj.setdefault("classification_dimension", None)
    obj.setdefault("filters", [])
    obj.setdefault("collector_parameters", {})
    obj.setdefault("steps", [])

    steps: List[Dict[str, Any]] = []
    for step in obj["steps"]:
        step = dict(step)
        if "description" not in step:
            sid = step.get("step_id", "step")
            agent = step.get("agent", "Agent")
            step["description"] = f"Шаг '{sid}' выполняется агентом {agent}."
        steps.append(step)

    obj["steps"] = steps
    return obj

In [50]:
from typing import List, Dict, Any, Literal
from psycopg2.extensions import connection as PGConnection

# По каким колонкам умеем брать DISTINCT
DistinctColumn = Literal["publication_year", "domain", "field", "subfield", "topic", "journal"]


class DBTool:
    """Обёртка над PostgreSQL для получения DISTINCT-значений
    из таблицы с научными публикациями.

    По умолчанию используем таблицу 'articles',
    но при необходимости можно передать другое имя (например, 'articles').
    """

    def __init__(self, conn: PGConnection, table_name: str = "publications") -> None:
        self.conn = conn
        self.table_name = table_name

    def list_distinct_values(self, column: DistinctColumn) -> List[Any]:
        """Вернуть список DISTINCT-значений для указанного столбца."""
        if column not in ("publication_year", "domain", "field", "subfield", "topic", "journal"):
            raise ValueError(f"Неподдерживаемый столбец для DISTINCT: {column}")

        order_clause = "ORDER BY publication_year" if column == "publication_year" else ""
        sql = f"SELECT DISTINCT {column} FROM {self.table_name} {order_clause};"

        with self.conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()

        return [row[0] for row in rows]

    def load_all_distinct_values(self) -> Dict[str, List[Any]]:
        """Загрузить словари всех поддерживаемых колонок."""
        result: Dict[str, List[Any]] = {}
        for col in ("publication_year", "domain", "field", "subfield", "topic", "journal"):
            result[col] = self.list_distinct_values(col)  # type: ignore[arg-type]
        return result


def run_collector_query(conn: PGConnection, sql_template: str, params: Dict[str, Any]):
    """Простейший Collector: подставляет именованные параметры и выполняет SQL.

    В Planner'е плейсхолдеры задаются как :param, а psycopg2 ожидает формат %(param)s,
    поэтому здесь делаем минимальный препроцессинг.
    """

    def convert_placeholders(sql: str) -> str:
        res = sql
        for name in params.keys():
            res = res.replace(f":{name}", f"%({name})s")
        return res

    sql = convert_placeholders(sql_template)

    with conn.cursor() as cur:
        cur.execute(sql, params)
        rows = cur.fetchall()

    return rows

In [52]:
import os
import json
import requests
from pydantic import ValidationError

BASE_URL = os.getenv("LITELLM_BASE_URL", "http://a6k2.dgx:34000/v1")
API_KEY = os.getenv("LITELLM_API_KEY", "sk-1yvtuYMQN37uRpXQe44qrA")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen3-32b")



PLANNER_SYSTEM_PROMPT = """Вы – агент Planner в многоагентной системе анализа научных публикаций.

В системе есть следующие агенты:
- Planner (вы): анализируете запрос пользователя, определяете цель, выбираете тип анализа и формируете план.
- Classifier: по вашему указанию определяет категории/кластеры (например, domain, field, subfield, topic) или дополняет метаданные публикаций.
- Collector: строит SQL-запросы к базе данных и извлекает нужные агрегированные данные.
- Analyst: на основе данных от Collector строит визуализации (графики, таблицы) и пишет текстовый аналитический вывод.

У нас есть PostgreSQL-БД с публикациями. В таблице articles содержатся, как минимум, следующие поля
(вы можете использовать только их, не придумывайте несуществующие столбцы):
- id
- title
- doi
- publication_year
- abstract
- citated_by_count
- journal
- domain
- field
- subfield
- topic
(если для ответа нужны другие поля, формулируйте это как желание, но НЕ используйте их в SQL-шаблоне).

ВАЖНО:
- Ваша задача – СДЕЛАТЬ ПЛАН, а НЕ выполнять SQL и не строить графики.
- Вы всегда возвращаете строго один JSON-объект, соответствующий схеме PlannerOutput.
- НЕ добавляйте пояснений, комментариев, маркдаун или текст вне JSON.
- НЕ НУЖНО возвращать JSON Schema или описание модели.
- НУЖНО вернуть КОНКРЕТНЫЙ ПЛАН с заполненными значениями полей PlannerOutput.


1. Сначала определите user_intent:
   - описать данные (describe_dataset),
   - изучить тренд по годам (trend_over_time),
   - сравнить группы (compare_groups),
   - найти топ-k сущностей (top_k_entities),
   - изучить связь между величинами (correlation_or_relationship),
   - получить просто отфильтрованную таблицу (raw_table_view),
   - либо другое (other).

2. Сформулируйте question_rewrite – точную техническую постановку задачи.

3. Решите, нужен ли Classifier (requires_classifier):
   - Если запрос затрагивает годы, домены, поля, подобласти, темы, журналы или другие сущности, которые должны быть выбраны из словарей допустимых значений – ставьте requires_classifier = true.
   - Если можно обойтись без выбора по словарю (например, чистая агрегация по всем публикациям без фильтров) – можно requires_classifier = false.

4. Заполните classification_dimensions:
   - Укажите, по каким полям таблицы articles Classifier должен подбирать значения: publication_year, domain, field, subfield, topic, journal, cited_by_count.
   - Например, если нужно выбрать годы 2018–2022 и темы, связанные с 'graph neural networks', то:
     classification_dimensions = ["publication_year", "topic"].

5. Заполните filters – текстовое описание фильтров на русском (без конкретных технических деталей SQL), например:
   - "Годы публикаций от 2018 до 2022 включительно"
   - "Темы, связанные с graph neural networks"
   - "Публикации в подобластях computer vision и natural language processing"

6. Составьте collector_sql_template:
   - Используйте ТОЛЬКО таблицу articles и поля, перечисленные выше.
   - НЕ подставляйте конкретные значения, только плейсхолдеры вида :start_year, :end_year, :topics, :domains и т.п.
   - Если вы ожидаете, что Classifier вернёт список значений, используйте IN (:topics), IN (:subfields), IN (:domains).
   - Пример: 
     SELECT publication_year, COUNT(*) AS n_papers
     FROM articles
     WHERE publication_year BETWEEN :start_year AND :end_year
       AND topic IN (:topics)
     GROUP BY publication_year
     ORDER BY publication_year;

7. Заполните collector_parameters:
   - Ключ = имя плейсхолдера без двоеточия (start_year, end_year, topics).
   - Значение = описание того, ЧТО должен подставить Classifier.
   - Пример:
     {
       "start_year": "Начальный год диапазона publication_year, выбранный Classifier'ом",
       "end_year": "Конечный год диапазона publication_year, выбранный Classifier'ом",
       "topics": "Список значений поля topic, выбранный Classifier'ом для описания тематики 'graph neural networks'"
     }

8. Заполните required_parameters:
   - Список всех имён параметров, которые должны быть заполнены перед выполнением SQL.
   - Например: ["start_year", "end_year", "topics"].

9. Определите expected_result_format:
   - "table" – если пользователю нужна только таблица;
   - "bar_chart" или "line_chart" – если речь о сравнении или динамике;
   - "histogram" – если важно распределение;
   - "text_summary" – чисто текстовый ответ;
   - "table_and_summary" или "chart_and_summary" – комбинированные варианты.

10. Заполните analyst_instructions:
   - Опишите, какие именно величины посчитать и как интерпретировать результат.
   - Если нужен график, явно укажите, какие поля по осям X и Y и что считать сериями.

11. Заполните steps:
   - Опишите цепочку действий от Planner к другим агентам, указывая agent для каждого шага.
   - Используйте step_id вроде "plan", "classify", "collect", "analyze".
   - В input_from указывайте список step_id предыдущих шагов, на которые опирается текущий шаг.

ЕЩЁ РАЗ: ответ должен быть СТРОГО одним JSON-объектом без пояснений.

Пример НЕПРАВИЛЬНОГО ответа (так делать нельзя):

{
  "title": "PlannerOutput",
  "type": "object",
  "properties": {
    "user_intent": { "type": "string", "enum": [...] },
    ...
  },
  "required": [...]
}

"""

import json
import requests
from pydantic import ValidationError

CLASSIFIER_SYSTEM_PROMPT = """Вы – агент Classifier в многоагентной системе анализа научных публикаций.

Входные данные, которые вам даёт Planner и окружающая система:
- Исходный запрос пользователя (user_query).
- Техническая переформулировка запроса (question_rewrite).
- Список полей таблицы articles, по которым нужно подобрать значения (classification_dimensions):
  publication_year, domain, field, subfield, topic, journal.
- Список параметров, которые нужно заполнить (required_parameters), например:
  ['start_year', 'end_year', 'topics'].
- Для каждого поля из classification_dimensions – список допустимых значений:
  - possible_publication_years: список целых чисел;
  - possible_domains: список строк;
  - possible_fields: список строк;
  - possible_subfields: список строк;
  - possible_topics: список строк;
  - possible_journals: список строк.
- SQL-шаблон Collector'а (collector_sql_template) и словарь описаний параметров (collector_parameters).

ВАЖНО:
- Вы НЕ пишете SQL и НЕ меняете SQL-шаблон.
- Ваша задача – заполнить параметры, которые указаны в required_parameters, используя ТОЛЬКО значения из переданных списков.
- Ответ должен быть СТРОГО одним JSON-объектом, соответствующим схеме ClassifierOutput.
"""


def call_classifier_llm(context: ClassifierContext) -> ClassifierOutput:
    """Вызов LLM-классификатора.

    context – объект ClassifierContext с:
      - user_query, question_rewrite,
      - required_parameters, classification_dimensions,
      - possible_* словарями значений,
      - collector_sql_template и collector_parameters.
    """
    url = f"{BASE_URL}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    user_content = json.dumps(context.model_dump(), ensure_ascii=False)

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": CLASSIFIER_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": (
                    "Ниже дан контекст для классификации в формате JSON. "
                    "Используйте его, чтобы заполнить параметры согласно инструкции.\\n\\n"
                    + user_content
                ),
            },
        ],
        "response_format": {"type": "json_object"},
        "temperature": 0.1,
    }

    resp = requests.post(url, headers=headers, json=payload, timeout=100)
    resp.raise_for_status()
    data = resp.json()

    # Пытаемся сначала взять reasoning_content, если модель его заполняет
    message = data["choices"][0]["message"]
    raw = message.get("reasoning_content") or message.get("content")
    if raw is None:
        raise RuntimeError(f"В ответе нет ни 'reasoning_content', ни 'content': {data}")

    try:
        obj = json.loads(raw)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Classifier вернул невалидный JSON: {e}\\nRaw: {raw!r}")

    try:
        return ClassifierOutput.model_validate(obj)
    except ValidationError as e:
        raise RuntimeError(f"ClassifierOutput не прошёл валидацию Pydantic: {e}")




def call_planner_llm(user_query: str) -> PlannerOutput:
    """
    Вызывает LLM (Qwen3-32B через LiteLLM/OpenAI-совместимый endpoint),
    возвращает объект PlannerOutput.
    """
    url = f"{BASE_URL}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": PLANNER_SYSTEM_PROMPT},
            {"role": "user", "content": user_query},
        ],
        # Просим модель вернуть строго JSON-объект
        "response_format": {"type": "json_object"},
        "temperature": 0.1,
    }

    resp = requests.post(url, headers=headers, json=payload, timeout=100)
    resp.raise_for_status()
    data = resp.json()
    # >>> ключевая строка: берём reasoning_content
    raw = data["choices"][0]["message"].get("reasoning_content")
    if raw is None:
        raise RuntimeError(f"В ответе нет поля 'reasoning_content': {data}")

    # reasoning_content — это строка с JSON
    import json
    try:
        obj = json.loads(raw)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Невалидный JSON в reasoning_content: {e}\nRaw: {raw!r}")

    # адаптер к вашей схеме, см. ниже
    obj = adapt_planner_json(obj)

    #print('проверка обжекта:')
    #print(obj)

    try:
        return PlannerOutput.model_validate(obj)
    except ValidationError as e:
        raise RuntimeError(f"PlannerOutput не прошёл валидацию Pydantic: {e}")


In [54]:
EXAMPLE_QUERIES = [
    # describe_dataset
    "Опиши, какие данные есть в нашей базе научных публикаций: диапазон лет, "
    "основные домены и поля, какие журналы и конференции встречаются чаще всего.",

    # trend_over_time
    "Построй динамику числа публикаций по теме deep learning с 2010 по 2024 год "
    "и прокомментируй тренды.",

    # compare_groups
    "Сравни динамику числа публикаций по computer vision и natural language processing "
    "с 2021 по 2024 год и выведи их на одном графике.",

    # top_k_entities
    #"Найди топ-20 авторов по количеству публикаций по теме transformers и выведи таблицу "
    #"с числом статей и средним числом цитирований.",

    # correlation_or_relationship
    "Исследуй связь между годом публикации и числом цитирований для статей по теме "
    "deep learning: падает ли цитируемость у более новых статей?",

    # raw_table_view
    "Покажи таблицу статей по теме graph neural networks, опубликованных с 2018 по 2022 год",

    # other (сложный комбинированный)
    #"Определи ключевые темы (topics) внутри домена artificial intelligence, "
    #"построй по ним динамику числа публикаций за период 2000–2024 и оцени, "
    #"какие темы сейчас растут быстрее всего.",
]

In [56]:
import psycopg2
from psycopg2.extras import execute_values

# ---- Параметры подключения к серверу PostgreSQL ----
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
TARGET_DB = "papers_db"


# ---------- 1. Создать БД, если её ещё нет ----------
def ensure_database():
    conn = psycopg2.connect(
        dbname="postgres",
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
    )
    conn.autocommit = True
    with conn.cursor() as cur:
        cur.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (TARGET_DB,))
        exists = cur.fetchone() is not None
        if not exists:
            print(f"Создаю базу данных {TARGET_DB}...")
            cur.execute(f"CREATE DATABASE {TARGET_DB};")
        else:
            print(f"База данных {TARGET_DB} уже существует.")
    conn.close()


# ---------- 2. Подключение к целевой БД ----------
def get_target_connection():
    return psycopg2.connect(
        dbname=TARGET_DB,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
    )


# ---------- 3. Таблица articles ----------
def create_table(conn):
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS articles (
        id SERIAL PRIMARY KEY,
        doi VARCHAR(255) UNIQUE NOT NULL,
        abstract TEXT,
        title TEXT NOT NULL,
        publication_year INT,
        cted_by_count INT,
        journal TEXT,
        domain TEXT,
        field TEXT,
        subfield TEXT,
        topic TEXT
    );
    """
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
    conn.commit()
    print("Таблица articles готова.")


# ---------- 3a. Гарантированно добавить новые колонки (на случай старой версии таблицы) ----------
def migrate_articles_add_new_columns(conn):
    with conn.cursor() as cur:
        cur.execute("""
            ALTER TABLE articles
                ADD COLUMN IF NOT EXISTS domain   TEXT,
                ADD COLUMN IF NOT EXISTS field    TEXT,
                ADD COLUMN IF NOT EXISTS subfield TEXT,
                ADD COLUMN IF NOT EXISTS topic    TEXT;
        """)
    conn.commit()
    print("Колонки domain, field, subfield, topic добавлены (если их не было).")
    
# ---------- 4. Заполнение / обновление данных ----------
def insert_sample_data(conn):
    # Базовые записи
    base_rows = [
        (
            "10.1000/xyz123",
            "We propose a new numerical method for solving large sparse linear systems arising in scientific computing.",
            "A Novel Iterative Method for Sparse Linear Systems",
            2020,
            35,
            "Journal of Computational Mathematics",
            "Mathematics",
            "Applied Mathematics",
            "Numerical Linear Algebra",
            "Sparse Linear Systems",
        ),
        (
            "10.1016/j.apm.2021.005",
            "The paper studies optimization algorithms for training deep neural networks with constraints.",
            "Constrained Optimization Methods in Deep Learning",
            2021,
            52,
            "Applied Mathematical Modelling",
            "Computer Science",
            "Artificial Intelligence",
            "Optimization for Deep Learning",
            "Constrained Optimization",
        ),
        (
            "10.1145/3394486.3403211",
            "We introduce an efficient graph-based recommendation system and evaluate it on real-world datasets.",
            "Graph-Based Recommender Systems at Scale",
            2019,
            120,
            "ACM Transactions on Information Systems",
            "Computer Science",
            "Information Retrieval",
            "Recommender Systems",
            "Graph-Based Recommendation",
        ),
        (
            "10.1109/TIT.2022.3141234",
            "This work analyzes error-correcting codes for modern communication channels with fading.",
            "Error-Correcting Codes for Fading Channels",
            2022,
            18,
            "IEEE Transactions on Information Theory",
            "Engineering",
            "Electrical Engineering",
            "Coding Theory",
            "Error-Correcting Codes",
        ),
        (
            "10.1093/biomet/asab012",
            "We discuss Bayesian approaches to modeling high-dimensional time series in econometrics.",
            "Bayesian Models for High-Dimensional Time Series",
            2021,
            44,
            "Biometrika",
            "Statistics",
            "Bayesian Statistics",
            "Time Series Analysis",
            "Bayesian Time Series",
        ),
        (
            "10.1137/19M1276416",
            "The article explores fast algorithms for large-scale eigenvalue problems in scientific computing.",
            "Fast Algorithms for Large-Scale Eigenvalue Problems",
            2019,
            67,
            "SIAM Journal on Scientific Computing",
            "Mathematics",
            "Numerical Analysis",
            "Eigenvalue Problems",
            "Large-Scale Eigenproblems",
        ),
    ]
    # Метаданные по тематикам
    topic_meta = {
        "Machine Learning": (
            "Computer Science",
            "Artificial Intelligence",
            "Machine Learning",
            "Supervised and Unsupervised Learning",
        ),
        "Numerical Analysis": (
            "Mathematics",
            "Applied Mathematics",
            "Numerical Analysis",
            "Numerical Methods",
        ),
        "Graph Theory": (
            "Mathematics",
            "Discrete Mathematics",
            "Graph Theory",
            "Graph Algorithms",
        ),
        "Optimization": (
            "Mathematics",
            "Optimization",
            "Convex and Nonlinear Optimization",
            "Optimization Methods",
        ),
        "Data Mining": (
            "Computer Science",
            "Data Science",
            "Data Mining",
            "Pattern Discovery",
        ),
        "Computer Vision": (
            "Computer Science",
            "Artificial Intelligence",
            "Computer Vision",
            "Image Recognition",
        ),
        "Reinforcement Learning": (
            "Computer Science",
            "Artificial Intelligence",
            "Reinforcement Learning",
            "Sequential Decision Making",
        ),
        "Cryptography": (
            "Computer Science",
            "Security",
            "Cryptography",
            "Modern Cryptographic Protocols",
        ),
        "Complex Networks": (
            "Physics",
            "Statistical Physics",
            "Complex Networks",
            "Network Dynamics",
        ),
        "Statistical Inference": (
            "Statistics",
            "Statistical Inference",
            "Parametric and Nonparametric Inference",
            "Estimation and Hypothesis Testing",
        ),
    }

    topics = list(topic_meta.keys())

    journals = [
        "Journal of Machine Learning Research",
        "Numerical Algorithms",
        "Discrete Mathematics",
        "Optimization Letters",
        "Data Mining and Knowledge Discovery",
        "IEEE Transactions on Neural Networks",
        "Pattern Recognition Letters",
        "Journal of Cryptology",
        "Network Science",
        "Annals of Statistics",
    ]

    generated_rows = []
    for i in range(1, 51):
        topic_name = topics[(i - 1) % len(topics)]
        domain, field, subfield, topic_label = topic_meta[topic_name]
        journal = journals[(i - 1) % len(journals)]
        publication_year = 2005 + (i % 20)
        cted_by_count = (i * 7) % 200

        doi = f"10.1234/sim.{publication_year}.{i:03d}"
        title = f"Simulated Study {i} on {topic_name}"
        abstract = (
            f"This simulated article ({i}) discusses methods and experiments related to "
            f"{topic_name.lower()} with applications in applied mathematics and computer science."
        )

        generated_rows.append(
            (
                doi,
                abstract,
                title,
                publication_year,
                cted_by_count,
                journal,
                domain,
                field,
                subfield,
                topic_label,
            )
        )

    rows = base_rows + generated_rows

    # ВАЖНО: используем DO UPDATE, чтобы заполнить колонки для уже существующих строк
    insert_sql = """
    INSERT INTO articles (
        doi,
        abstract,
        title,
        publication_year,
        cted_by_count,
        journal,
        domain,
        field,
        subfield,
        topic
    )
    VALUES %s
    ON CONFLICT (doi) DO UPDATE SET
        abstract         = EXCLUDED.abstract,
        title            = EXCLUDED.title,
        publication_year = EXCLUDED.publication_year,
        cted_by_count    = EXCLUDED.cted_by_count,
        journal          = EXCLUDED.journal,
        domain           = EXCLUDED.domain,
        field            = EXCLUDED.field,
        subfield         = EXCLUDED.subfield,
        topic            = EXCLUDED.topic;
    """

    with conn.cursor() as cur:
        execute_values(cur, insert_sql, rows)
    conn.commit()
    print(f"Добавлено/обновлено {len(rows)} записей.")


# ---------- 5. Проверочный вывод ----------
def show_data(conn, limit=15):
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT id, doi, title, publication_year, cted_by_count, journal,
                   domain, field, subfield, topic
            FROM articles
            ORDER BY id
            LIMIT %s;
            """,
            (limit,),
        )
        for row in cur.fetchall():
            print(row)


ensure_database()
conn = get_target_connection()
try:
    create_table(conn)
    migrate_articles_add_new_columns(conn)
    insert_sample_data(conn)
    print("Первые строки в таблице:")
    show_data(conn, limit=15)
finally:
    conn.close()

База данных papers_db уже существует.
Таблица articles готова.
Колонки domain, field, subfield, topic добавлены (если их не было).
Добавлено/обновлено 56 записей.
Первые строки в таблице:
(1, '10.1000/xyz123', 'A Novel Iterative Method for Sparse Linear Systems', 2020, 35, 'Journal of Computational Mathematics', 'Mathematics', 'Applied Mathematics', 'Numerical Linear Algebra', 'Sparse Linear Systems')
(2, '10.1016/j.apm.2021.005', 'Constrained Optimization Methods in Deep Learning', 2021, 52, 'Applied Mathematical Modelling', 'Computer Science', 'Artificial Intelligence', 'Optimization for Deep Learning', 'Constrained Optimization')
(3, '10.1145/3394486.3403211', 'Graph-Based Recommender Systems at Scale', 2019, 120, 'ACM Transactions on Information Systems', 'Computer Science', 'Information Retrieval', 'Recommender Systems', 'Graph-Based Recommendation')
(4, '10.1109/TIT.2022.3141234', 'Error-Correcting Codes for Fading Channels', 2022, 18, 'IEEE Transactions on Information Theory', '

In [58]:
conn = get_target_connection()

In [60]:


for i, q in enumerate(EXAMPLE_QUERIES, start=1):
        print("=" * 80)
        print(f"Запрос #{i}: {q}")
        print("-" * 80)

        planner_output = call_planner_llm(q)

        # Красивый вывод результата Planner’а
        print(
            json.dumps(
                planner_output.model_dump(mode="json"),
                ensure_ascii=False,
                indent=2,
            )
        )
        # 2. Словари значений из БД
        #    Здесь предполагается, что у тебя уже есть открытое соединение conn
        #    к нужной БД (например, через get_target_connection()).
        #    Если таблица называется 'articles', то поставь table_name='articles'.
        dbtool = DBTool(conn, table_name="articles")  # или "articles"
        possible_values = dbtool.load_all_distinct_values()
        
        classifier_output = None
        rows = None
        
        if planner_output.requires_classifier:
            # 3. Собираем контекст и вызываем Classifier
            classifier_ctx = build_classifier_context(user_query, planner_output, possible_values)
            classifier_output = call_classifier_llm(classifier_ctx)
        
            print("\\n=== ClassifierOutput ===")
            print(classifier_output.model_dump(mode="json"))
        
            # 4. Collector: выполняем SQL с подставленными параметрами
            rows = run_collector_query(
                conn,
                planner_output.collector_sql_template,
                classifier_output.resolved_parameters,
            )
        else:
            # Если Classifier не нужен, просто выполняем запрос без доп. параметров
            rows = run_collector_query(
                conn,
                planner_output.collector_sql_template,
                {},
            )
        
        print("\\n=== Первые строки результата Collector'а ===")
        print(rows[:5] if rows is not None else None)
        print()  # пустая строка между примерами

Запрос #1: Опиши, какие данные есть в нашей базе научных публикаций: диапазон лет, основные домены и поля, какие журналы и конференции встречаются чаще всего.
--------------------------------------------------------------------------------
{
  "user_intent": "describe_dataset",
  "question_rewrite": "Описать структуру и содержимое базы данных научных публикаций, включая диапазон лет, основные домены, поля, а также наиболее часто встречающиеся журналы и конференции.",
  "requires_classifier": false,
  "classification_dimension": null,
  "filters": [],
  "collector_sql_template": "SELECT MIN(publication_year) AS min_year, MAX(publication_year) AS max_year, COUNT(*) AS total_papers, COUNT(DISTINCT domain) AS unique_domains, COUNT(DISTINCT field) AS unique_fields, COUNT(DISTINCT journal) AS unique_journals FROM articles;",
  "collector_parameters": {},
  "required_parameters": [],
  "expected_result_format": "table_and_summary",
  "analyst_instructions": "Представить агрегированные метрики

RuntimeError: PlannerOutput не прошёл валидацию Pydantic: 1 validation error for PlannerOutput
filters
  Input should be a valid list [type=list_type, input_value="Годы публика...ые с 'deep learning'", input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type